In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import random

def next_batch(num_split, feature, labels):
    ind = np.arange(0, len(feature))
    np.random.shuffle(ind)
    ind = ind[:num_split]
    feature_shuffle = [feature[i].flatten() for i in ind]
    labels_shuffle = [labels[i] for i in ind]

    return np.asarray(feature_shuffle), np.asarray(labels_shuffle)

x_train = np.load("../saved_data/x_train.npy", allow_pickle=True)
x_test = np.load("../saved_data/x_test.npy", allow_pickle=True)
y_train = np.load("../saved_data/y_train.npy", allow_pickle=True)
y_test = np.load("../saved_data/y_test.npy", allow_pickle=True)

# Parameters
learning_rate = 0.0005
training_epochs = 150
batch_size = 32
display_step = 10

x = tf.placeholder(tf.float32, [None, 22500])
y = tf.placeholder(tf.float32, [None, 2])

W1 = tf.Variable(tf.random_normal([22500, 256], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([256]), name='b1')
W2 = tf.Variable(tf.random_normal([256, 2], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([2]), name='b2')

hidden_out = tf.add(tf.matmul(x, W1), b1)
hidden_out = tf.nn.relu(hidden_out)

log = tf.matmul(hidden_out, W2) + b2

pred = tf.nn.softmax(tf.add(tf.matmul(hidden_out, W2), b2))

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=log))
#optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# acc_ = tf.summary.scalar('acc', accuracy)
# loss_summary = tf.summary.scalar('loss', cost)


with tf.Session() as sess:
    #writer = tf.summary.FileWriter('../logs/NeuralNetwork', sess.graph)
    sess.run(tf.global_variables_initializer())

    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(x_train)/batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = next_batch(batch_size, x_train, y_train)
            _, c, acc = sess.run([optimizer, cost, accuracy], feed_dict={x: batch_xs, y: batch_ys})
            avg_cost += c / total_batch
        if (epoch+1) % display_step == 0:
#             writer.add_summary(acc_summary, epoch)
#             writer.add_summary(loss_, epoch)
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost), "acc=", "{:.5}".format(acc))
    #writer.close()
    print("Optimization Finished!")

    #saver = tf.train.Saver()
    #saver.save(sess, "../model/NeuralNetwork/model")

    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: x_test, y: y_test}))

Epoch: 0010 cost= 0.799914403 acc= 0.5625
Epoch: 0020 cost= 0.786937141 acc= 0.5625
Epoch: 0030 cost= 0.752236465 acc= 0.375
Epoch: 0040 cost= 0.695084761 acc= 0.5
Epoch: 0050 cost= 0.566327640 acc= 0.40625
Epoch: 0060 cost= 0.497871676 acc= 0.40625
Epoch: 0070 cost= 0.448900242 acc= 0.53125
Epoch: 0080 cost= 0.396513545 acc= 0.9375
Epoch: 0090 cost= 0.362843340 acc= 1.0
Epoch: 0100 cost= 0.664206511 acc= 0.375
Epoch: 0110 cost= 0.704688092 acc= 0.59375
